# Precipitation measurements

Example of homemade rain gauge precipitation measurements with the help of kids toy-buckets and some bayesian mathematics.

See https://en.wikipedia.org/wiki/Rain_gauge

Measurements were done overnight between 24.-25.7.2020.

## Weather station measurements
Espoo, Nuuksio, <15 km from the measurement location  
Espoo, Tapiola, <15 km from the measurement location  
https://www.ilmatieteenlaitos.fi/havaintojen-lataus

## Problem definition

The aim is to estimate precipitation based on simple measurement setup.
Two buckets (blue and yellow) were left outside overnight to gather rain precipitation.
The diameter of the bucket was measured a few times with a tape measure.
At the morning next day, the buckets were weighted with and without the precipitated water.
Scale was tared between the measurements and the buckets were placed approximately center.

Scale used for the weighting has approximately absolute accuracy of +/- 2 grams and the result is rounded to the closest 1 gram.
Tape measure results were inferred from the photograph and are given +/- 2 centimeter accuracy.

In [1]:
import numpy as np
import pandas as pd

## FMI Measurement data

In [2]:
fmi_data = pd.read_csv("./FMI_DATA/precipitation.csv", index_col=0)
fmi_data

,Precipitation overnight (mm)
Nuuksio,1.350000
Tapiola,4.066667


# Data

In [3]:
import pickle

In [4]:
with open("DATA/measurement_data.pickle", "rb") as f:
    data = pickle.load(f)

In [5]:
data.keys()

dict_keys(['weight_bucket', 'weigth_bucket_water', 'diameter', 'radius', 'wedge_outer', 'wedge_inner', 'wedge_length', 'density_water'])

# Engineering approximation --> mean it is

In [6]:
from helper_functions import calculate_bucket_area

In [7]:
area = calculate_bucket_area(
    data["radius"].mean(),
    data["wedge_inner"].mean(),
    data["wedge_outer"].mean(),
    data["wedge_length"].mean(),
)
area

blue      0.020834
yellow    0.021341
dtype: float64

In [8]:
weight_water = data["weigth_bucket_water"].mean() - data["weight_bucket"].mean()
weight_water

blue      41.666667
yellow    39.666667
dtype: float64

In [9]:
# (grams/1000) / density_water * 1000 --> grams / density_water
volume_water_litre = weight_water / data["density_water"]
volume_water_litre

blue      0.041708
yellow    0.039706
dtype: float64

In [10]:
precipitation = volume_water_litre / area
precipitation

blue      2.001923
yellow    1.860596
dtype: float64

In [11]:
print(f"Precipitation: {precipitation.mean():.2f} mm")

Precipitation: 1.93 mm
